In [1]:
import math
from math import log, sqrt, exp
import numpy as np
import scipy.stats as si
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.optimize import fmin
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import warnings

In [2]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

%matplotlib ipympl

In [3]:
SPX_spot=3662.45
SPY_spot=366.02

In [4]:
df_spx = pd.read_csv("SPX_options.csv")
df_spy = pd.read_csv("SPY_options.csv")
df_rates = pd.read_csv("zero_rates_20201201.csv")

In [5]:
df_spx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072 entries, 0 to 2071
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            2072 non-null   int64  
 1   exdate          2072 non-null   int64  
 2   cp_flag         2072 non-null   object 
 3   strike_price    2072 non-null   int64  
 4   best_bid        2072 non-null   float64
 5   best_offer      2072 non-null   float64
 6   exercise_style  2072 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 113.4+ KB


In [6]:
df_spy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            1400 non-null   int64  
 1   exdate          1400 non-null   int64  
 2   cp_flag         1400 non-null   object 
 3   strike_price    1400 non-null   int64  
 4   best_bid        1400 non-null   float64
 5   best_offer      1400 non-null   float64
 6   exercise_style  1400 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 76.7+ KB


In [7]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    45 non-null     int64  
 1   days    45 non-null     int64  
 2   rate    45 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.2 KB


#### Note that there are only 3 unique expiry dates and all the data is as of 1 day (there is no time series)

In [8]:
df_spx["exdate"].value_counts()

exdate
20201218    822
20210115    738
20210219    512
Name: count, dtype: int64

In [9]:
df_spy["exdate"].value_counts()

exdate
20210115    556
20201218    510
20210219    334
Name: count, dtype: int64

In [10]:
df_spx["date"].value_counts()

date
20201201    2072
Name: count, dtype: int64

In [11]:
df_spy["date"].value_counts()

date
20201201    1400
Name: count, dtype: int64

# Data Cleaning

In [12]:
# These look to be annualized rates

df_rates

,date,days,rate
0,20201201,7,0.102280
1,20201201,13,0.114128
2,20201201,49,0.216480
3,20201201,77,0.220707
4,20201201,104,0.219996
5,20201201,139,0.218208
6,20201201,167,0.216468
7,20201201,195,0.215228
8,20201201,286,0.212862
9,20201201,377,0.214085


In [13]:
df_rates["date"] = pd.to_datetime(df_rates["date"], format="%Y%m%d")

In [14]:
df_rates["rate_decimal"] = df_rates["rate"] / 100

In [15]:
df_rates = df_rates.drop(["date"], axis=1)
df_rates.set_index("days", inplace=True)
df_rates = df_rates.reindex(np.arange(df_rates.index.min(), df_rates.index.max() + 1))
df_rates = df_rates.interpolate(method="linear")

In [16]:
df_rates

,rate,rate_decimal
days,,
7,0.102280,0.001023
8,0.104255,0.001043
9,0.106229,0.001062
10,0.108204,0.001082
11,0.110179,0.001102
...,...,...
3572,0.955703,0.009557
3573,0.955906,0.009559
3574,0.956109,0.009561


In [17]:
df_spx

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style
0,20201201,20201218,C,100000,3547.6,3570.5,E
1,20201201,20201218,C,200000,3447.6,3470.5,E
2,20201201,20201218,C,300000,3347.7,3370.6,E
3,20201201,20201218,C,400000,3247.7,3270.6,E
4,20201201,20201218,C,500000,3147.7,3170.6,E
...,...,...,...,...,...,...,...
2067,20201201,20210219,P,5000000,1333.1,1350.5,E
2068,20201201,20210219,P,5100000,1431.8,1454.7,E
2069,20201201,20210219,P,5200000,1531.7,1554.6,E
2070,20201201,20210219,P,5300000,1631.5,1654.4,E


In [18]:
df_spx["date"] = pd.to_datetime(df_spx["date"], format="%Y%m%d")
df_spx["exdate"] = pd.to_datetime(df_spx["exdate"], format="%Y%m%d")

In [19]:
df_spx["days_to_expiry"] = (df_spx["exdate"] - df_spx["date"]) / pd.Timedelta(days=1)
df_spx["years_to_expiry"] = df_spx["days_to_expiry"] / 365

In [20]:
df_spx["mid_price"] = 0.5 * (df_spx["best_bid"] + df_spx["best_offer"])

In [21]:
df_spx["strike_price"] = df_spx["strike_price"] / 1000

In [22]:
df_spx["options_type"] = df_spx["cp_flag"].map(lambda x: "call" if x == "C" else "put")

In [23]:
df_spx=df_spx.merge(df_rates,left_on="days_to_expiry",right_index=True)

In [24]:
df_spx

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal
0,2020-12-01,2020-12-18,C,100.0,3547.6,3570.5,E,17.0,0.046575,3559.05,call,0.125500,0.001255
1,2020-12-01,2020-12-18,C,200.0,3447.6,3470.5,E,17.0,0.046575,3459.05,call,0.125500,0.001255
2,2020-12-01,2020-12-18,C,300.0,3347.7,3370.6,E,17.0,0.046575,3359.15,call,0.125500,0.001255
3,2020-12-01,2020-12-18,C,400.0,3247.7,3270.6,E,17.0,0.046575,3259.15,call,0.125500,0.001255
4,2020-12-01,2020-12-18,C,500.0,3147.7,3170.6,E,17.0,0.046575,3159.15,call,0.125500,0.001255
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,2020-12-01,2021-02-19,P,5000.0,1333.1,1350.5,E,80.0,0.219178,1341.80,put,0.220628,0.002206
2068,2020-12-01,2021-02-19,P,5100.0,1431.8,1454.7,E,80.0,0.219178,1443.25,put,0.220628,0.002206
2069,2020-12-01,2021-02-19,P,5200.0,1531.7,1554.6,E,80.0,0.219178,1543.15,put,0.220628,0.002206
2070,2020-12-01,2021-02-19,P,5300.0,1631.5,1654.4,E,80.0,0.219178,1642.95,put,0.220628,0.002206


In [25]:
df_spy

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style
0,20201201,20201218,C,25000,340.74,341.20,A
1,20201201,20201218,C,50000,315.75,316.21,A
2,20201201,20201218,C,75000,290.75,291.21,A
3,20201201,20201218,C,80000,285.75,286.21,A
4,20201201,20201218,C,85000,280.75,281.21,A
...,...,...,...,...,...,...,...
1395,20201201,20210219,P,480000,115.03,115.93,A
1396,20201201,20210219,P,485000,120.08,120.92,A
1397,20201201,20210219,P,490000,125.07,125.92,A
1398,20201201,20210219,P,495000,129.98,130.92,A


In [26]:
df_spy["date"] = pd.to_datetime(df_spy["date"], format="%Y%m%d")
df_spy["exdate"] = pd.to_datetime(df_spy["exdate"], format="%Y%m%d")

In [27]:
df_spy["days_to_expiry"] = (df_spy["exdate"] - df_spy["date"]) / pd.Timedelta(days=1)
df_spy["years_to_expiry"] = df_spy["days_to_expiry"] / 365

In [28]:
df_spy["mid_price"] = 0.5 * (df_spy["best_bid"] + df_spy["best_offer"])

In [29]:
df_spy["strike_price"] = df_spy["strike_price"] / 1000

In [30]:
df_spy["options_type"] = df_spy["cp_flag"].map(lambda x: "call" if x == "C" else "put")

In [31]:
df_spy

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type
0,2020-12-01,2020-12-18,C,25.0,340.74,341.20,A,17.0,0.046575,340.970,call
1,2020-12-01,2020-12-18,C,50.0,315.75,316.21,A,17.0,0.046575,315.980,call
2,2020-12-01,2020-12-18,C,75.0,290.75,291.21,A,17.0,0.046575,290.980,call
3,2020-12-01,2020-12-18,C,80.0,285.75,286.21,A,17.0,0.046575,285.980,call
4,2020-12-01,2020-12-18,C,85.0,280.75,281.21,A,17.0,0.046575,280.980,call
...,...,...,...,...,...,...,...,...,...,...,...
1395,2020-12-01,2021-02-19,P,480.0,115.03,115.93,A,80.0,0.219178,115.480,put
1396,2020-12-01,2021-02-19,P,485.0,120.08,120.92,A,80.0,0.219178,120.500,put
1397,2020-12-01,2021-02-19,P,490.0,125.07,125.92,A,80.0,0.219178,125.495,put
1398,2020-12-01,2021-02-19,P,495.0,129.98,130.92,A,80.0,0.219178,130.450,put


In [32]:
df_spy=df_spy.merge(df_rates,left_on="days_to_expiry",right_index=True)

# Calculate implied volatility using brentq

https://stackoverflow.com/questions/61289020/fast-implied-volatility-calculation-in-python

https://medium.com/@polanitzer/implied-volatility-in-python-compute-the-volatilities-implied-by-option-prices-observed-in-the-e2085c184270

https://github.com/jieren123/stock-implied-volatility-using-root-finding/blob/master/Brent.py

# Version 1

In [77]:
# Calculate the option price using Black-Scholes formula
def calculate_option_price(S0: float,
                           K: float,
                           r: float,
                           sigma: float,
                           T: float,
                           option_type="call"):

    if option_type not in ["call", "put"]:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    if T == 0:
        if option_type == "call":
            return max(S0 - K, 0)
        else:
            return max(K - S0, 0)


    d1 = (np.log(S0/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == "call":
        return S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)

In [78]:
# Function to calculate implied volatility using Brent's method
def calculate_implied_volatility(S0: float,
                                 K: float,
                                 r: float,
                                 mid_price: float,
                                 T: float,
                                 option_type="call"):
    
   # Define the objective function to minimize
    def objective_function(sigma):
        return mid_price - calculate_option_price(S0, K, r, sigma, T, option_type)
    
    # Define the volatility bounds for brentq
    low, high = 1e-6, 10.0  # Set more reasonable bounds for volatility
    
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html
    # Use brentq to find the root (implied volatility)
    try:
        implied_vol = brentq(objective_function, low, high, xtol=1e-12)
    except ValueError:  # If no solution is found in the range, return NaN
        implied_vol = np.nan

    return implied_vol

In [79]:
# Applying the function on the DataFrame
df_spx["implied_vol"] = df_spx.apply(
    lambda x: calculate_implied_volatility(
        S0=SPX_spot,
        K=x["strike_price"],
        r=x["rate_decimal"],
        T=x["years_to_expiry"],
        mid_price=x["mid_price"], 
        option_type=x["options_type"]
    ), axis=1
)

In [80]:
df_spx

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal,implied_vol
0,2020-12-01,2020-12-18,C,100.0,3547.6,3570.5,E,17.0,0.046575,3559.05,call,0.125500,0.001255,NaN
1,2020-12-01,2020-12-18,C,200.0,3447.6,3470.5,E,17.0,0.046575,3459.05,call,0.125500,0.001255,NaN
2,2020-12-01,2020-12-18,C,300.0,3347.7,3370.6,E,17.0,0.046575,3359.15,call,0.125500,0.001255,NaN
3,2020-12-01,2020-12-18,C,400.0,3247.7,3270.6,E,17.0,0.046575,3259.15,call,0.125500,0.001255,NaN
4,2020-12-01,2020-12-18,C,500.0,3147.7,3170.6,E,17.0,0.046575,3159.15,call,0.125500,0.001255,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,2020-12-01,2021-02-19,P,5000.0,1333.1,1350.5,E,80.0,0.219178,1341.80,put,0.220628,0.002206,0.341058
2068,2020-12-01,2021-02-19,P,5100.0,1431.8,1454.7,E,80.0,0.219178,1443.25,put,0.220628,0.002206,0.370768
2069,2020-12-01,2021-02-19,P,5200.0,1531.7,1554.6,E,80.0,0.219178,1543.15,put,0.220628,0.002206,0.387810
2070,2020-12-01,2021-02-19,P,5300.0,1631.5,1654.4,E,80.0,0.219178,1642.95,put,0.220628,0.002206,0.403519


In [37]:
#df_spx.to_csv(path_or_buf="df_spx.csv")

In [73]:
SPY_spot

366.02

In [38]:
# Applying the function on the DataFrame
df_spy["implied_vol"] = df_spy.apply(
    lambda x: calculate_implied_volatility(
        S0=SPY_spot,
        K=x["strike_price"],
        r=x["rate_decimal"],
        T=x["years_to_expiry"],
        mid_price=x["mid_price"], 
        option_type=x["options_type"]
    ), axis=1
)

In [39]:
df_spy

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal,implied_vol
0,2020-12-01,2020-12-18,C,25.0,340.74,341.20,A,17.0,0.046575,340.970,call,0.125500,0.001255,NaN
1,2020-12-01,2020-12-18,C,50.0,315.75,316.21,A,17.0,0.046575,315.980,call,0.125500,0.001255,NaN
2,2020-12-01,2020-12-18,C,75.0,290.75,291.21,A,17.0,0.046575,290.980,call,0.125500,0.001255,NaN
3,2020-12-01,2020-12-18,C,80.0,285.75,286.21,A,17.0,0.046575,285.980,call,0.125500,0.001255,NaN
4,2020-12-01,2020-12-18,C,85.0,280.75,281.21,A,17.0,0.046575,280.980,call,0.125500,0.001255,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,2020-12-01,2021-02-19,P,480.0,115.03,115.93,A,80.0,0.219178,115.480,put,0.220628,0.002206,0.365322
1396,2020-12-01,2021-02-19,P,485.0,120.08,120.92,A,80.0,0.219178,120.500,put,0.220628,0.002206,0.376933
1397,2020-12-01,2021-02-19,P,490.0,125.07,125.92,A,80.0,0.219178,125.495,put,0.220628,0.002206,0.387244
1398,2020-12-01,2021-02-19,P,495.0,129.98,130.92,A,80.0,0.219178,130.450,put,0.220628,0.002206,0.395489


# Version 2

In [70]:
def impliedVolatility(S0, K, r, price, T, payoff):
    try:
        if (payoff.lower() == 'call'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalCall(S0, K, r, x, T),
                                1e-12, 20.0)
        elif (payoff.lower() == 'put'):
            impliedVol = brentq(lambda x: price -
                                BlackScholesLognormalPut(S0, K, r, x, T),
                                1e-12, 20.0)
        else:
            raise NameError('Payoff type not recognized')
    except Exception:
        impliedVol = np.nan

    return impliedVol


def BlackScholesLognormalCall(S0, K, r, sigma, T):
    d1 = (np.log(S0/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesLognormalPut(S0, K, r, sigma, T):
    d1 = (np.log(S0/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)

In [71]:
# Applying the function on the DataFrame
df_spy["implied_vol"] = df_spy.apply(
    lambda x: impliedVolatility(
        S0=SPY_spot,
        K=x["strike_price"],
        r=x["rate_decimal"],
        T=x["years_to_expiry"],
        price=x["mid_price"], 
        payoff=x["options_type"]
    ), axis=1
)

In [72]:
df_spy

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal,implied_vol
0,2020-12-01,2020-12-18,C,25.0,340.74,341.20,A,17.0,0.046575,340.970,call,0.125500,0.001255,NaN
1,2020-12-01,2020-12-18,C,50.0,315.75,316.21,A,17.0,0.046575,315.980,call,0.125500,0.001255,NaN
2,2020-12-01,2020-12-18,C,75.0,290.75,291.21,A,17.0,0.046575,290.980,call,0.125500,0.001255,NaN
3,2020-12-01,2020-12-18,C,80.0,285.75,286.21,A,17.0,0.046575,285.980,call,0.125500,0.001255,NaN
4,2020-12-01,2020-12-18,C,85.0,280.75,281.21,A,17.0,0.046575,280.980,call,0.125500,0.001255,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,2020-12-01,2021-02-19,P,480.0,115.03,115.93,A,80.0,0.219178,115.480,put,0.220628,0.002206,0.365322
1396,2020-12-01,2021-02-19,P,485.0,120.08,120.92,A,80.0,0.219178,120.500,put,0.220628,0.002206,0.376933
1397,2020-12-01,2021-02-19,P,490.0,125.07,125.92,A,80.0,0.219178,125.495,put,0.220628,0.002206,0.387244
1398,2020-12-01,2021-02-19,P,495.0,129.98,130.92,A,80.0,0.219178,130.450,put,0.220628,0.002206,0.395489


# Version 3

In [62]:
class ImpliedVolatility_Brent(object):
    def __init__(self, S, K, r, T, option_type):
        self.S = S
        self.K = K
        self.r = r
        self.T = T
        self.option_type = option_type

    def bsmValue(self, sigma):
        d1 = (log(self.S / self.K) + (self.r + 0.5 * sigma ** 2) * self.T) / (sigma * sqrt(self.T))
        d2 = d1 - sigma * sqrt(self.T)

        if self.option_type.lower() == 'call':
            return self.S * si.norm.cdf(d1) - self.K * exp(-self.r * self.T) * si.norm.cdf(d2)

        elif self.option_type.lower() == 'put':
            return self.K * exp(-self.r * self.T) *(1 - si.norm.cdf(d2)) - self.S * (1 - si.norm.cdf(d1))

        else:
            raise TypeError('the option_type argument must be either "call" or "put"')

    def get_implied_volatilities(self):
        f = lambda sigma: self.bsmValue(sigma) 
        impv = brentq(f, 0.01, 10.0, xtol = 1e-12)
        return impv

In [63]:
def calculate_implied_vol(row):
    # Extract variables from the row
    S = SPY_spot
    K = row['strike_price']
    r = row['rate_decimal']
    T = row['years_to_expiry']
    option_type = row['options_type']
    market_price = row['mid_price']

    # Instantiate the ImpliedVolatility_Brent class
    iv_calculator = ImpliedVolatility_Brent(S, K, r, T, option_type)

    # try:
    # Get the implied volatility using the market price
    implied_vol = iv_calculator.get_implied_volatilities()
    # except ValueError:
    #     # In case of any errors, return NaN
    #     implied_vol = float('nan')

    return implied_vol

In [64]:
# Apply the implied volatility calculation to each row
df_spy['implied_vol'] = df_spy.apply(calculate_implied_vol,axis=1)

ValueError: f(a) and f(b) must have different signs

In [65]:
df_spy

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal,implied_vol
0,2020-12-01,2020-12-18,C,25.0,340.74,341.20,A,17.0,0.046575,340.970,call,0.125500,0.001255,-0.293420
1,2020-12-01,2020-12-18,C,50.0,315.75,316.21,A,17.0,0.046575,315.980,call,0.125500,0.001255,-0.109823
2,2020-12-01,2020-12-18,C,75.0,290.75,291.21,A,17.0,0.046575,290.980,call,0.125500,0.001255,-0.093371
3,2020-12-01,2020-12-18,C,80.0,285.75,286.21,A,17.0,0.046575,285.980,call,0.125500,0.001255,-0.089480
4,2020-12-01,2020-12-18,C,85.0,280.75,281.21,A,17.0,0.046575,280.980,call,0.125500,0.001255,-0.085748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,2020-12-01,2021-02-19,P,480.0,115.03,115.93,A,80.0,0.219178,115.480,put,0.220628,0.002206,-0.068408
1396,2020-12-01,2021-02-19,P,485.0,120.08,120.92,A,80.0,0.219178,120.500,put,0.220628,0.002206,-0.007770
1397,2020-12-01,2021-02-19,P,490.0,125.07,125.92,A,80.0,0.219178,125.495,put,0.220628,0.002206,-0.016690
1398,2020-12-01,2021-02-19,P,495.0,129.98,130.92,A,80.0,0.219178,130.450,put,0.220628,0.002206,-0.025116


# Using fmin to calculate implied vol

https://medium.com/@polanitzer/implied-volatility-in-python-compute-the-volatilities-implied-by-option-prices-observed-in-the-e2085c184270

In [44]:
S0 = 60
K = 65
rf = 0.08
T = 0.25
P0 = 8
C0 = 8

In [45]:
def ImpliedVolatilityCall(sigma, S0, K, rf, T, C0):
    d1 = (np.log(S0/K) + (rf + 0.5*sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Black-Scholes formula for Call option price
    call_price = S0 * norm.cdf(d1) - K * np.exp(-rf * T) * norm.cdf(d2)
    
    # The objective is to minimize the squared difference between the market price and model price
    return (call_price - C0)**2

In [46]:
def ImpliedVolatilityPut(sigma, S0, K, rf, T, P0):
    d1 = (np.log(S0/K) + (rf + 0.5*sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Black-Scholes formula for Put option price
    put_price = K * np.exp(-rf * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    
    # The objective is to minimize the squared difference between the market price and model price
    return (put_price - P0)**2

In [47]:
# Use fmin to minimize the difference and find the implied volatility
initial_guess = 0.3
result = fmin(ImpliedVolatilityCall, x0=initial_guess, args=(S0, K, rf, T, C0))

print(f"Implied Volatility for Call Option: {result[0]}")

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 17
         Function evaluations: 34
Implied Volatility for Call Option: 0.7951757812500002


In [48]:
# Use fmin to minimize the difference and find the implied volatility
initial_guess = 0.3

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html
result = fmin(ImpliedVolatilityPut, x0=initial_guess, args=(S0, K, rf, T, P0))

print(f"Implied Volatility: {result[0]}")

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 14
         Function evaluations: 28
Implied Volatility: 0.48439453125000015


In [49]:
# Unified function for both call and put options
def ImpliedVolatility(sigma, S0, K, rf, T, market_price, option_type="call"):
    d1 = (np.log(S0/K) + (rf + 0.5*sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        # Black-Scholes formula for Call option price
        option_price = S0 * norm.cdf(d1) - K * np.exp(-rf * T) * norm.cdf(d2)
    elif option_type == "put":
        # Black-Scholes formula for Put option price
        option_price = K * np.exp(-rf * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    # The objective is to minimize the squared difference between market price and model price
    return (option_price - market_price) ** 2

In [50]:
# Function to calculate implied volatility using fmin
def calculate_implied_volatility(S0, K, rf, T, market_price, option_type="call"):
    # Initial guess for volatility
    initial_guess = 0.3

    # Minimize the objective function using fmin
    result = fmin(
        ImpliedVolatility, x0=initial_guess, args=(S0, K, rf, T, market_price, option_type), disp=False
    )

    return result[0]  # Return the implied volatility

In [51]:
df_spy["implied_vol"] = df_spy.apply(
    lambda x: calculate_implied_volatility(
        S0=SPY_spot,  # Spot price (same for all rows)
        K=x["strike_price"],  # Strike price from DataFrame
        rf=x["rate_decimal"],  # Risk-free rate from DataFrame
        T=x["years_to_expiry"],  # Time to expiration from DataFrame
        market_price=x["mid_price"],  # Market price from DataFrame
        option_type=x["options_type"]  # 'call' or 'put' from DataFrame
    ), axis=1
)

In [52]:
df_spy

,date,exdate,cp_flag,strike_price,best_bid,best_offer,exercise_style,days_to_expiry,years_to_expiry,mid_price,options_type,rate,rate_decimal,implied_vol
0,2020-12-01,2020-12-18,C,25.0,340.74,341.20,A,17.0,0.046575,340.970,call,0.125500,0.001255,0.300000
1,2020-12-01,2020-12-18,C,50.0,315.75,316.21,A,17.0,0.046575,315.980,call,0.125500,0.001255,0.300000
2,2020-12-01,2020-12-18,C,75.0,290.75,291.21,A,17.0,0.046575,290.980,call,0.125500,0.001255,0.300000
3,2020-12-01,2020-12-18,C,80.0,285.75,286.21,A,17.0,0.046575,285.980,call,0.125500,0.001255,0.300000
4,2020-12-01,2020-12-18,C,85.0,280.75,281.21,A,17.0,0.046575,280.980,call,0.125500,0.001255,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,2020-12-01,2021-02-19,P,480.0,115.03,115.93,A,80.0,0.219178,115.480,put,0.220628,0.002206,0.365332
1396,2020-12-01,2021-02-19,P,485.0,120.08,120.92,A,80.0,0.219178,120.500,put,0.220628,0.002206,0.376934
1397,2020-12-01,2021-02-19,P,490.0,125.07,125.92,A,80.0,0.219178,125.495,put,0.220628,0.002206,0.387246
1398,2020-12-01,2021-02-19,P,495.0,129.98,130.92,A,80.0,0.219178,130.450,put,0.220628,0.002206,0.395508
